# Helper functions and scripts for downloading data from AWS S3 Buckets

In [2]:
import pathlib
import sys
import os
import boto3
# Add path to parent/root directory
parent = os.path.dirname(os.getcwd())
sys.path.insert(0, parent)

# Reload modules after every code change/save
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
AWS_ACCESS_KEY_ID=""
AWS_SECRET_ACCESS_KEY=""
AWS_SESSION_TOKEN=""

session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,  
    region_name='us-east-1'
)

# Create an S3 client
s3 = session.client('s3')

# List files in a specific S3 bucket directory
def list_files_in_bucket(bucket, prefix=''):
    contents = []
    response = s3.list_objects(Bucket=bucket, Prefix=prefix)
    if 'Contents' in response:
        for item in response['Contents']:
            contents.append(item['Key'])
    return contents

# Download a file from an S3 bucket
def download_file_from_bucket(bucket, file_name, local_path):
    s3.download_file(bucket, file_name, local_path)


In [3]:
# # Download images from s3 bucket for label studio annotations that you connect with s3. Images dont get downloaded in the export

# bucket_name = 'surface-damage-raw-cards'
# prefix = 'Gum Stains'

# dataset_path = './dataset_315'

# for l in os.listdir(f'{dataset_path}/labels'):
#     img_file=f"{l[:l.rfind('.')]}.jpg"
#     f=f"{prefix}/{l[:l.rfind('.')]}.jpg"
#     print(img_file)
#     print(f)
#     break
#     # download_file_from_bucket(bucket_name, f, f'{dataset_path}/images/{img_file}')

In [11]:
# Download images not already downloaded/annotated

bucket_name = 'surface-damage-raw-cards'
datasets = ['gum_stain_on_back', 'gum_stains'] # gum_stain_on_back or gum_stains
category = "FULL_CARD"
prefix = f'CATEGORIZED/{datasets[1]}/CATEGORIZED/{category}'

dataset_paths = [] # Directory where images already downloaded/annotated
images_bucket = list_files_in_bucket(bucket_name, prefix=prefix)

output_path = f'./dataset_more_balanced_new/{category}' # THe new directory of images that will be created
os.makedirs(output_path, exist_ok=True) 

already_downloaded_files = []
# for d in dataset_paths:
#     for l in os.listdir(f'{d}/labels'):
#         already_downloaded_files.append(f"{l[:l.rfind('.')]}.jpg")

num = 0
for f in images_bucket[1:]:
    img_file=(f[f.rfind('/')+1:])
    if img_file not in already_downloaded_files:
        num+=1
        download_file_from_bucket(bucket_name, f, f'{output_path}/{img_file}')
print(num)

180


In [5]:
# datasets_to_check = ['./dataset_more_balanced_548/labels', './dataset_val_balanced/labels']
# TYPES = ['FULL_CARD', 'PARTIAL_CARD', 'LIGHT', 'SMALL', 'RECTANGULAR', 'WHITE_MARKS']
# COLORS = ['BLUE', 'GREEN', 'ORANGE', 'RED', 'YELLOW']

# datasets = ['gum_stains', 'gum_stain_on_back'] # gum_stain_on_back or gum_stains

# def card_stats(dataset):
#     bucket_name = 'surface-damage-raw-cards'

#     prefix = f'CATEGORIZED/{dataset}/CATEGORIZED'

#     s3_bucket_counts = {t: {c: 0 for c in COLORS + ['total']} for t in TYPES} 

#     already_downloaded_files = []
#     dataset_counts = {t: {c: 0 for c in COLORS + ['total']} for t in TYPES}
#     if len(datasets_to_check) > 0:
#         for data in datasets_to_check:
#             for l in os.listdir(data):
#                 already_downloaded_files.append(f"{l[:l.rfind('.')]}.jpg")

#     for t in TYPES:
#         sum_type = 0
#         dataset_type = 0
#         for c in COLORS:
#             sum_color = 0
#             dataset_color = 0
#             for f in list_files_in_bucket(bucket=bucket_name, prefix=f'{prefix}/{t}/{c}'):
#                 img_file=(f[f.rfind('/')+1:])
#                 if img_file in already_downloaded_files:
#                     dataset_color += 1
#                 sum_color += 1
#             s3_bucket_counts[t][c] = sum_color
#             dataset_counts[t][c] = dataset_color
#             sum_type += sum_color
#             dataset_type += dataset_color
#         s3_bucket_counts[t]['total'] = sum_type
#         dataset_counts[t]['total'] = dataset_type

#     return s3_bucket_counts, dataset_counts

# total_counts = {t: {c: 0 for c in COLORS + ['total']} for t in TYPES}

# # Function to display statistics from the sum_countså
# def display_summary_statistics(counts):
#     print("Statistics for folders:")
#     print("-" * 40)
    
#     total_files = 0
    
#     for t in counts:
#         print(f"\nType: {t} Total: {counts[t]['total']}\n")
#         total_files += counts[t]['total']
        
#         for c in COLORS:
#             print(f"Color: {c}  Total: {counts[t][c]}")
#         print()
    
#     print("-" * 40)
#     print(f"Total number of files: {total_files}")

# # Display statistics
# if len(datasets_to_check) > 0:
#     for d in datasets:
#         print(d)
#         s3_bucket_counts, dataset_counts = card_stats(d)

#         # Add dataset_counts to total_counts
#         for t in TYPES:
#             for c in COLORS + ['total']:
#                 total_counts[t][c] += dataset_counts[t][c]
#         # display_summary_statistics(sum_counts)
#         # display_summary_statistics(dataset_counts)

# display_summary_statistics(total_counts)

gum_stains
gum_stain_on_back
Statistics for folders:
----------------------------------------

Type: FULL_CARD Total: 132

Color: BLUE  Total: 10
Color: GREEN  Total: 28
Color: ORANGE  Total: 67
Color: RED  Total: 8
Color: YELLOW  Total: 19


Type: PARTIAL_CARD Total: 197

Color: BLUE  Total: 26
Color: GREEN  Total: 33
Color: ORANGE  Total: 86
Color: RED  Total: 3
Color: YELLOW  Total: 49


Type: LIGHT Total: 79

Color: BLUE  Total: 7
Color: GREEN  Total: 3
Color: ORANGE  Total: 47
Color: RED  Total: 12
Color: YELLOW  Total: 10


Type: SMALL Total: 45

Color: BLUE  Total: 0
Color: GREEN  Total: 3
Color: ORANGE  Total: 34
Color: RED  Total: 0
Color: YELLOW  Total: 8


Type: RECTANGULAR Total: 62

Color: BLUE  Total: 13
Color: GREEN  Total: 1
Color: ORANGE  Total: 34
Color: RED  Total: 3
Color: YELLOW  Total: 11


Type: WHITE_MARKS Total: 93

Color: BLUE  Total: 3
Color: GREEN  Total: 2
Color: ORANGE  Total: 71
Color: RED  Total: 8
Color: YELLOW  Total: 9

-------------------------------